# Part 1

In [1]:
# 03. import libraries
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import scipy

In [2]:
# import path
path = r'C:\Users\Bradley Allen\Desktop\Instacart Basket Analysis'

In [3]:
#import original customer data as a dataframe
df_cust  = pd.read_csv(r'C:\Users\Bradley Allen\Desktop\Instacart Basket Analysis\02 Data\Original Data\customers.csv')

In [4]:
df_cust.head()

,user_id,First Name,Surnam,Gender,STATE,Age,date_joined,n_dependants,fam_status,income
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Patricia,Hart,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Kenneth,Farley,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Michelle,Hicks,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Ann,Gilmore,Female,Maryland,26,1/1/2017,1,married,40374


In [5]:
df_cust.shape

(206209, 10)

In [6]:
df_cust.describe()

,user_id,Age,n_dependants,income
count,206209.000000,206209.000000,206209.000000,206209.000000
mean,103105.000000,49.501646,1.499823,94632.852548
std,59527.555167,18.480962,1.118433,42473.786988
min,1.000000,18.000000,0.000000,25903.000000
25%,51553.000000,33.000000,0.000000,59874.000000
50%,103105.000000,49.000000,1.000000,93547.000000
75%,154657.000000,66.000000,3.000000,124244.000000
max,206209.000000,81.000000,3.000000,593901.000000


# 04. Data Wrangling, rename columns

In [7]:
df_cust = df_cust.rename(columns = {'First Name':'first_name', 'Surnam':'last_name', 'Gender':'gender', 'STATE':'state', 'Age':'age'})

In [8]:
df_cust.dtypes

user_id          int64
first_name      object
last_name       object
gender          object
state           object
age              int64
date_joined     object
n_dependants     int64
fam_status      object
income           int64
dtype: object

# 05. data quality check -- check for mising values

In [9]:
df_cust.isnull().sum()

user_id             0
first_name      11259
last_name           0
gender              0
state               0
age                 0
date_joined         0
n_dependants        0
fam_status          0
income              0
dtype: int64

### only missing data is in the first name column... unlikely to need this in analysis, as observations can be uniquely
### identified with 'user_id'
### WILL DROP FIRST NAME COLUMN

In [10]:
df_cust = df_cust.drop('first_name', 1)

C:\Users\BRADLE~1\AppData\Local\Temp/ipykernel_144/1638187623.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  df_cust = df_cust.drop('first_name', 1)


### checking for duplicates

In [11]:
df_dup = df_cust[df_cust.duplicated()]

In [12]:
df_dup

,user_id,last_name,gender,state,age,date_joined,n_dependants,fam_status,income


### No fully duplicated rows detected

### Checking for Mixed data types

In [13]:
for col in df_cust.columns.tolist():
    weird = (df_cust[[col]].applymap(type) != df_cust[[col]].iloc[0].apply(type)).any(axis = 1)
    if len (df_cust[weird]) > 0:
        print (col)

### no mixed data types were found

# 06. combing customer data with the rest of prepared data

In [14]:
## importing prepared orders_products dataframe
df_ords_prods = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', '4.8_clean_aggregated_data.pkl'))

In [15]:
df_ords_prods.head()

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,first_order,product_id,add_to_cart_order,reordered,...,aisle_id,department_id,prices,_merge,max_order,loyalty_flag,user_avg_price,spending_habit,order_frequency,order_frequncy
0,2539329,1,1,2,8,NaN,True,196,1,0,...,77.0,7.0,9.0,both,10,New customer,5.175,Lower spender,NaN,NaN
2,2539329,1,1,2,8,NaN,True,12427,3,0,...,23.0,19.0,4.4,both,10,New customer,5.175,Lower spender,NaN,NaN
6,2398795,1,2,3,7,15.0,False,10258,2,0,...,117.0,19.0,3.0,both,10,New customer,5.175,Lower spender,NaN,Regular customer
12,473747,1,3,3,12,21.0,False,12427,2,1,...,23.0,19.0,4.4,both,10,New customer,5.175,Lower spender,Non-frequent customer,NaN
13,473747,1,3,3,12,21.0,False,10258,3,1,...,117.0,19.0,3.0,both,10,New customer,5.175,Lower spender,Non-frequent customer,NaN


In [16]:
df_ords_prods = df_ords_prods.drop('order_frequncy', 1)

C:\Users\BRADLE~1\AppData\Local\Temp/ipykernel_144/3693434549.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  df_ords_prods = df_ords_prods.drop('order_frequncy', 1)


In [17]:
df_ords_prods = df_ords_prods.drop('_merge', 1)

C:\Users\BRADLE~1\AppData\Local\Temp/ipykernel_144/582997468.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  df_ords_prods = df_ords_prods.drop('_merge', 1)


In [18]:
df_ords_prods = df_ords_prods.drop('reordered', 1)

C:\Users\BRADLE~1\AppData\Local\Temp/ipykernel_144/1000938628.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  df_ords_prods = df_ords_prods.drop('reordered', 1)


In [19]:
df_ords_prods.head()

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,first_order,product_id,add_to_cart_order,product_name,aisle_id,department_id,prices,max_order,loyalty_flag,user_avg_price,spending_habit,order_frequency
0,2539329,1,1,2,8,NaN,True,196,1,Soda,77.0,7.0,9.0,10,New customer,5.175,Lower spender,NaN
2,2539329,1,1,2,8,NaN,True,12427,3,Original Beef Jerky,23.0,19.0,4.4,10,New customer,5.175,Lower spender,NaN
6,2398795,1,2,3,7,15.0,False,10258,2,Pistachios,117.0,19.0,3.0,10,New customer,5.175,Lower spender,NaN
12,473747,1,3,3,12,21.0,False,12427,2,Original Beef Jerky,23.0,19.0,4.4,10,New customer,5.175,Lower spender,Non-frequent customer
13,473747,1,3,3,12,21.0,False,10258,3,Pistachios,117.0,19.0,3.0,10,New customer,5.175,Lower spender,Non-frequent customer


In [20]:
## merging customer df and ords_prods df
df_merge = df_ords_prods.merge(df_cust, on = 'user_id', indicator = True)

In [21]:
df_merge.head(70)

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,first_order,product_id,add_to_cart_order,product_name,...,order_frequency,last_name,gender,state,age,date_joined,n_dependants,fam_status,income,_merge
0,2539329,1,1,2,8,NaN,True,196,1,Soda,...,NaN,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423,both
1,2539329,1,1,2,8,NaN,True,12427,3,Original Beef Jerky,...,NaN,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423,both
2,2398795,1,2,3,7,15.0,False,10258,2,Pistachios,...,NaN,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423,both
3,473747,1,3,3,12,21.0,False,12427,2,Original Beef Jerky,...,Non-frequent customer,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423,both
4,473747,1,3,3,12,21.0,False,10258,3,Pistachios,...,Non-frequent customer,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423,both
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,2710558,3,5,0,17,12.0,False,28373,6,Original Rotisserie Chicken,...,NaN,Fry,Female,Arizona,33,7/28/2017,3,married,93240,both
66,1839752,3,7,0,15,7.0,False,39190,1,Vanilla Unsweetened Almond Milk,...,Frequent customer,Fry,Female,Arizona,33,7/28/2017,3,married,93240,both
67,1839752,3,7,0,15,7.0,False,1819,2,All Natural No Stir Creamy Almond Butter,...,Frequent customer,Fry,Female,Arizona,33,7/28/2017,3,married,93240,both
68,3160850,3,9,0,16,7.0,False,43961,4,Organic Peeled Whole Baby Carrots,...,Frequent customer,Fry,Female,Arizona,33,7/28/2017,3,married,93240,both


In [22]:
df_merge['_merge'].value_counts(dropna = False)

both          6480166
left_only           0
right_only          0
Name: _merge, dtype: int64

## merge was successful (indicated by the _merge only containing both)

In [23]:
df_merge.to_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'ords_prods_cust(4.9p1).pkl'))